In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz 
import os
import json
from pathlib import Path
import spacy
import en_core_web_sm
import nltk
import re
nlp = en_core_web_sm.load()
nlp.max_length = 6000000 

In [ ]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [ ]:
def clean_dataframe(dataframe, column_name):
    dataframe[column_name] = dataframe[column_name].str.lower()
    pattern = '|'.join(['<s>', '</s>',])
    dataframe[column_name] = dataframe[column_name].str.replace(pattern, '')
    dataframe[column_name] = dataframe[column_name].str.replace('\(s\)', 's') #reference: https://stackoverflow.com/questions/51440233/how-to-remove-the-values-which-are-in-parentheses-in-pandashttps://stackoverflow.com/questions/51440233/how-to-remove-the-values-which-are-in-parentheses-in-pandas
    return dataframe    

In [ ]:
query = "SELECT * FROM issues';"
with engine.connect() as conn:
    df = pd.read_sql(query, conn)
df

In [ ]:
query = "SELECT * FROM issues WHERE tableId = '3e1c53b4-5c01-46e2-bd72-5a338b5852f9';"

with engine.connect() as conn:
    df = pd.read_sql(query, conn)
    #clean_dataframe(df, 'table_content')
dic = {}

for row in df.itertuples():
    # converting JSON string to a list of lists of strings
    table = json.loads(row.table_content)
    headers = table[0]  # column headers  
    for header in headers:
        #header = lemmawordnet(header)
        #header = clean_header(header)
        #header = lemmaspacy(header)
        #header = re.sub(r'\(.*?\)', lambda x: ''.join(x.group(0).split()), header) # removing whitespace between parentheses (reference: https://stackoverflow.com/questions/34088489/how-to-remove-whitespace-inside-brackethttps://stackoverflow.com/questions/34088489/how-to-remove-whitespace-inside-bracket)
        header = " ".join(header.split())
        if header in dic:
            dic[header] += 1
        else:
            dic[header] = 1

my_list = [(header, count) for header, count in dic.items()]  # Converting to list
my_list.sort(key=lambda tup: tup[1], reverse=True)  # sorting the list

total_headers = 0
print(f"COUNT\tHEADER")
for item in my_list:
    total_headers += item[1]
    print(f"{item[1]}\t{item[0]}")
print()
print(f'Total headers: {total_headers}; unique headers: {len(my_list)}')

In [ ]:
df3 = pd.DataFrame(my_list)
df3.to_csv('headers4.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
for row in df.itertuples():
    table = json.loads(row.table_content)
    print(type(table))
    lst_element1 = "SPREAD"
    lst_element2 = "QUARTER SECTION"
    if lst_element1 in table[0][0] and lst_element2 in table[0][1]:
        print(row.tableId)

In [ ]:
for row in df.itertuples():
    table = json.loads(row.table_content)
    if table[0][len(table[0])-1] == "Topic":
        data = pd.DataFrame(table)
        data.to_csv(r"C:\Users\t1nipun\Desktop\PCMR\human-robot\Data_Analysis\csvs\\" + row.tableId + '.csv', encoding = 'utf-8-sig', index = False, header = None)
    else:
        table[0].extend(['VEC', 'GIS', 'Topic'])
        for i in range(1, len(table)-1):
            table[i].extend(['','',''])
        data = pd.DataFrame(table)
        data.to_csv(r"C:\Users\t1nipun\Desktop\PCMR\human-robot\Data_Analysis\csvs\\" + row.tableId + '.csv', encoding = 'utf-8-sig', index = False, header = None)

In [ ]:
!pip install opencv-python
!pip install camelot-py[all] 
# Please install ghostcript using this link: https://www.ghostscript.com/
import camelot
import tkinter
#tables = camelot.read_pdf('G:/Post Construction/george.pdf', pages = '7', line_scale=40, flag_size=True, copy_text=['v'],) # latice method
tables = camelot.read_pdf('G:/Post Construction/george.pdf', flavor = 'stream', edge_tol=500, pages = '6-38') # stream method
tables.export('scheduleA.csv', f='csv', compress=True)

In [ ]:
tables.export('scheduleA.csv', f='csv', compress=True) # json, excel, html
tables[0]

In [ ]:
tables[0].parsing_report

In [ ]:
tables[0].to_csv('george1.csv')
tables[0].df